# Nettoyage du jeu de données concernant l'étude sur les primes d'assurance

## Importation des librairies et du jeu de données

In [15]:
import pandas as pd

data = pd.read_csv("dataset.csv")

## Première description du jeu de données

Rappel concernant le jeu de données constitué des variables suivantes : 

- l’indice de masse corporel (bmi) : ça permet de donner un rapport entre la taille et le poids. 
- le sexe (sex): le genre de la personne qui contracte l'assurance, homme ou femme
- l’âge (age): l'âge du principal bénéficiaire
- le nombre d’enfant à charge (children) : Nombre d'enfant couverts par l'assurance
- smoker : fumeur ou non-fumeur
- région (region) : le zone résidentielle dans les US, nord-est, sud-est, sud-ouest, nord-ouest
- charges : la prime d’assurance facturée (cible)
​

In [16]:
display(data.shape)
display(data.dtypes)

(1338, 7)

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

Nous avons un jeu de données composé de 7 variables et de 1338 observations. 

Parmi ces variables, nous avons :
- deux variables quantitatives discrètes (age, children)
- deux variables quantitatives continues (bmi, charges)
- trois variables qualitatives nominales (sex, smoker, region)

On remarque qu'il n'y a pas un type de données anormal.

## Vérification et nettoyage du jeu de données

### Vérification et nettoyage globale

#### Données manquantes

In [17]:
display(data.info())
display(data.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


None

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

Il n'y a aucune valeur nulle dans le jeu de données.

#### Doublons

On va effectuer deux types de recherches de doublons:

- un premier pour les observations ayant des charges identiques
- un deuxième pour les observations ayant les mêmes valeurs, mis à part pour les charges dont on ne tient pas compte

In [18]:
duplicate_charges = data.loc[data[["charges"]].duplicated(keep=False),:]
display(duplicate_charges)

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.5631
581,19,male,30.59,0,no,northwest,1639.5631


On peut remarquer qu'il y a un doublon aux lignes indexées 195 et 581, que nous allons supprimer, ce qui portera notre nombre d'observations à 1337.

In [19]:
data.drop_duplicates(subset=["charges"], inplace=True, ignore_index=True)
display(data.shape)

(1337, 7)

In [20]:
duplicate_values = data.loc[data[["age", "sex", "bmi", "children", "smoker", "region"]].duplicated(keep=False),:]

display(duplicate_values)

,age,sex,bmi,children,smoker,region,charges
102,18,female,30.115,0,no,northeast,21344.84670
181,18,female,38.280,0,no,southeast,1631.82120
354,18,female,38.280,0,no,southeast,14133.03775
471,18,female,30.115,0,no,northeast,2203.47185


Pour cette seconde vérification on remarque deux paires d'observations identiques à l'exception des charges (102 / 471 et 181 / 354). On note d'ailleurs un fort écart concernant les charges, avec des charges quasiment dix fois plus élevées pour un élément de chaque paire.

En l'état, nous ne disposons pas des informations nécessaires pour expliquer cette différence.

### Vérification et nettoyage par variable

Nous allons désormer chercher les outliers, les valeurs aberrantes ou non conformes au jeu de données, parmi nos différentes variables.

Pour cela, on regarde premièrement les données générales telles que les valeurs maximales et minimales et les moyennes.

#### Variables quantitatives ("age", "bmi", "children", "charges")

In [21]:
data.describe()

,age,bmi,children,charges
count,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,30.663452,1.095737,13279.121487
std,14.044333,6.100468,1.205571,12110.359656
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.290000,0.000000,4746.344000
50%,39.000000,30.400000,1.000000,9386.161300
75%,51.000000,34.700000,2.000000,16657.717450
max,64.000000,53.130000,5.000000,63770.428010


En regardant ce tableau, on se rend compte que les charges et le bmi montent ont des valeurs atypiques concernant leurs maximales. 
Pour le bmi ça ne parait pas aberrant puisque la moyenne est proche de 30, en revanche ça l'est plus pour les charges ou le 3eme quartile est égale à 16 657 (une différence de 50 000).

In [22]:
third_quartile = data.loc[data["charges"]>16675,:]
display(third_quartile["charges"].describe())

count      334.000000
mean     31151.710338
std      10581.240592
min      16776.304050
25%      21345.811525
50%      28936.803060
75%      39727.090013
max      63770.428010
Name: charges, dtype: float64

En isolant les observations dont les charges sont supérieurs au 3ème quartile, on remarque que les valeurs sont assez étalées. 
Le 3ème quartile pour ce sous-échantillon est égale à 39 727. La valeur maximale s'apparente donc plus à une valeur atypique qu'à une valeur aberrante.

On va également jeter un oeil à l'index de ces lignes pour avoir un aperçu de l'ensemble des données pour ces valeurs.

In [23]:
high_bmi = data.loc[data['bmi']==53.13, :].index[0]
display("bmi élevé :")
display(data.iloc[high_bmi,:])

high_charges = data.loc[data['charges']==63770.428010, :].index[0]
display("charges élevées :")
display(data.iloc[high_charges,:])

'bmi élevé :'

age                18
sex              male
bmi             53.13
children            0
smoker             no
region      southeast
charges     1163.4627
Name: 1316, dtype: object

'charges élevées :'

age                  54
sex              female
bmi               47.41
children              0
smoker              yes
region        southeast
charges     63770.42801
Name: 543, dtype: object

On remarque que les valeurs sont effectivement suspectes, notamment la différence concernant les charges, mais pas totalement incohérentes. On pourra en apprendre plus lors de l'exploration des données, mais pour le moment elles ne sont pas inexploitables.

#### Variables qualitatives ("sex", "smoker", "region")

In [24]:
display(data["sex"].value_counts())
display(data["smoker"].value_counts())
display(data["region"].value_counts())

sex
male      675
female    662
Name: count, dtype: int64

smoker
no     1063
yes     274
Name: count, dtype: int64

region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64

On ne remarque pas de valeurs aberrantes parmi les variables qualitatives.

## Conclusion du nettoyage et chargement du jeu de données dans un fichier csv

Le nettoyage du jeu de données n'a pour le moment nécessité que la suppression d'une ligne de doublon. On exporte donc le jeu de données dans un nouveau fichier csv pour les analyser.

In [25]:
data.to_csv("dataset_cleaned.csv", index=False)